In [113]:
import pandas as pd
import os

In [114]:
mes_referencia="07"
ano_referencia="22"

In [115]:
def suma_mes(mes):
    mes_cadena=str(int(mes)+1)
    if len(mes_cadena)==1:
        mes_cadena="0"+ mes_cadena
    return mes_cadena

In [116]:
tramas=[]
for (root,dirs,files) in os.walk('TRAMAS'):
    for name in files:
        if name[0:1]=="M":
            dia=name[12:14]
            mes=name[15:17]
            ano=name[20:22]
            if (mes_referencia==mes and ano_referencia == ano) or (dia=="01" and suma_mes(mes_referencia)==mes):
                tramas.append(root + "\\"+ os.path.join(name))

In [133]:
# pandas concat csv files
df = pd.concat((pd.read_csv(f, sep=";", encoding="latin-1", header=None, skiprows=1) for f in tramas), ignore_index=False)
df.columns =  ["Tipo", "Estacion", "Fecha", "Hora_inicio", "Mezcla", "Premezcla", "Batch", "Receta", "Kg_corcho", "Minutos_pesada", "Kg_DU", "Tiempo_vaciado", "Tiempo_inyeccion_agua", "Kg_Agua", "Tiempo_mezcla_agua", "Tiempo_inyeccion_cola", "Kg_cola", "Tiempo_mezcla_cola", "Tiempo_vaciado_mezcla", "Molde_inicial_ext1", "Molde_final_ext1", "Receta", "Molde_inicial_ext2", "Molde_final_ext2", "Tapones_buenos_g1", "Tapones_malos_g1", "Tiempo_extraccion_g1", "Tapones_buenos_g2", "Tapones_malos_g2", "Tiempo_extraccion_g2", "Tapones_buenos_descartados_g1", "Tapones_buenos_descartados_g2", "nada_1", "nada_2"]

In [134]:
df.drop(columns=["Tipo", "Hora_inicio", "Premezcla", "Batch", "Receta", "Minutos_pesada", "Tiempo_vaciado", "Tiempo_inyeccion_agua", "Kg_Agua", "Tiempo_mezcla_agua", "Tiempo_inyeccion_cola", "Tiempo_mezcla_cola", "Tiempo_vaciado_mezcla", "Molde_inicial_ext1", "Molde_final_ext1", "Receta", "Molde_inicial_ext2", "Molde_final_ext2", "Tiempo_extraccion_g1", "Tiempo_extraccion_g2","nada_1", "nada_2"], inplace=True)	

In [135]:
df

,Estacion,Fecha,Mezcla,Kg_corcho,Kg_DU,Kg_cola,Tapones_buenos_g1,Tapones_malos_g1,Tapones_buenos_g2,Tapones_malos_g2,Tapones_buenos_descartados_g1,Tapones_buenos_descartados_g2
0,10,30/6/22,68,56135,3088,28069,6204,0,6160,44,0,0
1,10,30/6/22,69,56131,3088,28069,6204,0,6248,0,0,0
2,10,30/6/22,70,55908,3075,27957,6292,0,6292,0,0,0
3,10,1/7/22,1,55984,3080,28013,5940,0,5984,0,0,0
4,10,1/7/22,2,56139,3088,28069,5940,0,5940,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
137,15,31/7/22,55,50251,2915,25128,5104,0,5060,0,7,0
138,15,31/7/22,55,50251,2915,25128,5104,0,5060,0,7,0
139,15,31/7/22,55,50251,2915,25128,5104,0,5060,0,7,0
140,15,31/7/22,56,50381,2973,25199,4928,0,5060,0,23,0
